In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000


In [2]:
xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree
print( str(xl_dgree) )

sql = "SELECT * \
from day_zhibiao \
where \
    jx_days_ud60 > 10 and jx_days_ud60 < 60 and \
    jx_xl_60>-"+str(xl_dgree)+" and  \
    jx_xl_20>0 "
print(sql)
# and \
#     lj_fl_3_10>1 and lj_fl_5_20>1 
df = pd.read_sql_query(sql, conn)
df.shape

# ds_qf60_1 ：jx_days_ud60 > 20 and jx_days_ud60 < 40 and jx_xl_60>-"+str(xl_02dgree)+" and jx_xl_10>0 and jx_xl_20>0 
# ds_qf60_2 ：jx_days_ud60 > 15 and jx_days_ud60 < 60 and jx_xl_60>-"+str(xl_02dgree)+" and jx_xl_10>0 and jx_xl_20>0 
# ds_qf60_3 ：jx_days_ud60 > 10 and jx_days_ud60 < 60 and jx_xl_60>-"+str(xl_02dgree)+"                and jx_xl_20>0 


0.00523
SELECT * from day_zhibiao where     jx_days_ud60 > 10 and jx_days_ud60 < 60 and     jx_xl_60>-0.00523 and      jx_xl_20>0 


(294721, 62)

In [3]:
# 检视数据状态
col_list = ['ts_code', 'trade_date', 'close', 'pre_close', 'pct_change', 'volume', \
    'diff', 'bar',\
    'jx_days_ud60',\
    'jx_xl_250','jx_xl_120', 'jx_xl_60', \
    'jx_xl_20', 'jx_xl_10', 'jx_xl_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20', 'lj_fl_20_60', \
    'sz_max_5', 'sz_max_10', 'sz_max_20' ]
# col_list
df_qf60 = df[col_list].copy()
# df_qf60.info()

In [8]:
# 输出数据集

# 确定训练目标：1个月内上涨30%
df['test10_10'] = df['sz_max_10'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 确定输出列  
col_list_ds = [ 'pct_change',  \
    'diff', 'bar',\
    'jx_days_ud60',\
    'jx_xl_250','jx_xl_120', 'jx_xl_60', \
    'jx_xl_20', 'jx_xl_10', 'jx_xl_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20', 'lj_fl_20_60', \
    'test10_10', 'test20_20', 'test20_30' ]
df_ds = df[col_list_ds].copy()


In [ ]:
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', conn, index=False, if_exists='append', chunksize=2000) 

In [5]:
df[ df['sz_max_10']>10 ].shape, \
    df[ df['sz_max_20']>20 ].shape, df[ df['sz_max_20']>30 ].shape



((37488, 65), (21792, 65), (8404, 65))

In [6]:
df[ df['sz_max_20']>20 ]

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_change,volume,...,lj_fl_20_60,sz_sum_5,sz_sum_10,sz_sum_20,sz_max_5,sz_max_10,sz_max_20,test10_10,test20_20,test20_30
77,000002.SZ,20221124,16.03,16.17,15.76,16.02,15.49,0.53,3.4216,1316720.00,...,1.225400,17.8693,20.1264,17.0654,18.60170,20.9738,27.7154,True,True,False
78,000002.SZ,20221123,15.40,15.73,15.38,15.49,15.60,-0.11,-0.7051,467413.00,...,1.227530,19.4142,18.7912,20.3266,21.56230,23.8218,32.0852,True,True,True
79,000002.SZ,20221122,15.80,15.95,15.39,15.60,15.57,0.03,0.1927,828369.00,...,1.275150,19.6650,21.6315,19.6749,20.70510,22.9487,31.1538,True,True,True
80,000002.SZ,20221121,15.57,15.60,15.08,15.57,15.65,-0.08,-0.5112,763224.00,...,1.269930,9.8694,20.4500,23.1751,11.23960,22.0295,31.4066,True,True,True
147,000004.SZ,20221104,8.75,8.92,8.72,8.86,8.76,0.10,1.1416,11770.00,...,0.591250,5.9344,17.2272,13.8423,8.91648,18.2844,30.3612,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294702,689009.SH,20210818,64.39,66.13,64.02,65.59,64.50,1.09,1.6899,7386.42,...,0.779694,29.4674,17.4365,21.0203,31.23950,31.2395,31.2395,True,True,True
294703,689009.SH,20210817,65.13,66.33,64.10,64.50,65.13,-0.63,-0.9673,7509.06,...,0.794421,11.1517,17.8464,18.7235,11.20930,33.4574,33.4574,True,True,True
294704,689009.SH,20210816,64.00,65.93,63.12,65.13,63.80,1.33,2.0846,9409.06,...,0.811494,1.5026,16.4237,20.3536,1.33579,32.1664,32.1664,True,True,True
294705,689009.SH,20210812,66.73,69.80,66.20,67.76,67.99,-0.23,-0.3383,18661.90,...,0.817692,-3.8451,25.2919,25.6831,0.00000,27.0366,27.0366,True,True,False


In [29]:
# 输出今天的test数据集
today0 = '20230719'
sql_today = "SELECT * \
from day_zhibiao \
where trade_date='" + today0 + "' and \
jx_days_ud60 > 10 and jx_days_ud60 < 60 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql_today)


SELECT * from day_zhibiao where trade_date='20230719' and jx_days_ud60 > 10 and jx_days_ud60 < 60 and jx_xl_60>-0.00523 and  jx_xl_20>0 


In [30]:

df_today = pd.read_sql_query(sql_today, conn)
df_today.shape, 


((187, 62),)

In [31]:
df_today.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 62 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ts_code       187 non-null    object 
 1   trade_date    187 non-null    object 
 2   open          187 non-null    float64
 3   high          187 non-null    float64
 4   low           187 non-null    float64
 5   close         187 non-null    float64
 6   pre_close     187 non-null    float64
 7   change        187 non-null    float64
 8   pct_change    187 non-null    float64
 9   volume        187 non-null    float64
 10  value         187 non-null    float64
 11  id            187 non-null    int64  
 12  pct_upper     187 non-null    float64
 13  pct_down      187 non-null    float64
 14  pct_range     187 non-null    float64
 15  pct_st_range  187 non-null    float64
 16  st_high       187 non-null    float64
 17  st_low        187 non-null    float64
 18  xing          187 non-null    

In [32]:
col_list_ds = ['ts_code', 'trade_date', 'pct_change',  \
    'diff', 'bar',\
    'jx_days_ud60',\
    'jx_xl_250','jx_xl_120', 'jx_xl_60', \
    'jx_xl_20', 'jx_xl_10', 'jx_xl_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20', 'lj_fl_20_60']
df_out = df_today[col_list_ds].copy()
# 删除旧数据
sql_deldata = "truncate table ds_qf60_today"
# basic.exec_sql(conn, sql_deldata)
# 输出新数据
df_out.to_sql('ds_qf60_today', conn, index=False, if_exists='append', chunksize=2000) 

187

In [33]:
df_out

,ts_code,trade_date,pct_change,diff,bar,jx_days_ud60,jx_xl_250,jx_xl_120,jx_xl_60,jx_xl_20,jx_xl_10,jx_xl_5,jx_zs_5,jx_zs_10,jx_zs_20,jx_zs_60,lj_fl_1_3,lj_fl_3_10,lj_fl_5_20,lj_fl_20_60
0,000042.SZ,20230719,2.2546,-0.015513,0.023579,46,-0.000705,-0.000526,-0.001115,0.001732,0.001444,0.005525,1,1,2,0,1.040380,1.181330,1.003500,0.949894
1,000088.SZ,20230719,0.0000,-0.011260,0.017436,42,-0.000150,-0.000535,-0.001187,0.000720,0.001020,0.002442,4,5,9,0,0.914340,0.864374,0.821242,0.548799
2,000415.SZ,20230719,0.9302,-0.003997,0.006472,22,-0.000036,-0.000717,0.000610,0.000705,0.001861,0.003721,2,2,5,0,0.965013,1.242590,0.981429,0.592953
3,000507.SZ,20230719,0.7207,-0.012643,0.013830,27,-0.000266,-0.000146,-0.000503,0.000181,0.001264,0.003239,5,5,5,0,1.144840,1.141350,0.996740,0.685797
4,000552.SZ,20230719,0.3030,-0.006032,0.006326,12,-0.000972,0.000246,-0.000497,0.000305,0.001819,0.000605,1,9,8,0,1.029400,1.001330,0.924873,0.815219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,603889.SH,20230719,2.6992,0.063414,0.043690,36,0.000710,0.000803,-0.001700,0.001239,0.004335,0.001780,1,1,25,0,1.454690,0.777920,0.747658,0.673211
183,603950.SH,20230719,0.7175,-0.020589,0.036384,49,0.000400,-0.000473,-0.004717,0.001708,-0.002146,0.000523,2,1,2,0,0.930688,0.966351,1.417340,1.401470
184,605016.SH,20230719,-0.1625,-0.035743,0.167947,33,-0.000116,0.000000,-0.001907,0.004275,0.003984,0.003023,7,6,7,0,0.805576,0.707396,0.499216,1.368960
185,605180.SH,20230719,0.6955,-0.221590,0.158731,29,-0.001419,-0.001491,-0.002691,0.002830,0.002641,0.004175,3,14,10,0,1.277500,0.906585,0.638194,0.778451
